In [1]:
from electroviz.core.experiment import Experiment

exp = Experiment("E:/RG008/")

Parsing directories...
Loading NIDAQ data...
Loading bTsS data...
Loading Imec data...
Loading Kilosort data...
11145871
Aligning Syncs...
8681 8681
8677 8677
8681 8681
8677 8677
Creating Stimuli...
Creating Populations...
Creating Probes...
Done!


In [ ]:
import numpy as np
exp.kilosort[0].kilosort_dict.keys()
np.unique(exp.kilosort[0].kilosort_dict["spike_clusters"]).size

In [7]:
stimuli = exp.stimuli
pop0 = exp.populations[0]
pop0 = pop0.remove(pop0.units["total_spikes"] < 100)
pop0_sort = pop0.sort("peak_channel")

pop0_sort.plot_raster(stimuli[2])

In [9]:
probe = exp.probes[0]
probe.plot_CSD(stimuli[2], subtract=True)

In [10]:
from electroviz.core.kernel import SparseNoiseKernel
from electroviz.core.kernel import StaticGratingsKernel
from electroviz.viz.summary import UnitSummary
import matplotlib.pyplot as plt

population = pop0_sort
ctx_range = (40, 80)
lgn_range = (225, 390)

for ctx_idx, ctx_unit in enumerate(population[slice(*ctx_range)]):

    kernels = [SparseNoiseKernel(ctx_unit, stimuli[0]), 
               SparseNoiseKernel(ctx_unit, stimuli[1]), 
               StaticGratingsKernel(ctx_unit, stimuli[2]), 
               StaticGratingsKernel(ctx_unit, stimuli[3])]

    file_path = "G:/Shared drives/Neuropixels/Figures/RG008/UnitSummary/CTX/"
    file_name = "Probe1_Unit{0}".format(str(ctx_idx + ctx_range[0]))

    UnitSummary(ctx_unit, stimuli[:4], kernels, save_path=file_path+file_name)

    plt.close("all")

C:\Ryan\GitHub\electroviz\electroviz\viz\psth.py:47: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(lims[0])


KeyboardInterrupt: 